# IMPORTS

Model imports

In [2]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

c:\Users\victo\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KFold cross validation imports

In [4]:
import splitfolders
from sklearn.model_selection import KFold, StratifiedKFold

# Device

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Image transformation

In [6]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

# Data loader

In [7]:
splitfolders.ratio("../DataSegmented", output="../DataTrainTest", seed=1337, ratio=(.8, 0, .2), group_prefix=None, move=False)

Copying files: 2424 files [00:01, 1502.22 files/s]


Separation no KFold cross

# KFold cross validation

In [8]:
data_dir = '../DataSegmented'

In [9]:
folds = StratifiedKFold(n_splits=10)

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]

NameError: name 'digits' is not defined

In [10]:
train_path='../DataTrainTest/train/'
test_path='../DataTrainTest/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [11]:
for i, (images,labels) in enumerate(train_loader):
    print(images)

tensor([[[[-0.6784, -0.8118, -0.8275,  ..., -0.8431, -0.8196, -0.7020],
          [-0.8118, -0.9294, -0.9294,  ..., -0.9451, -0.9294, -0.8118],
          [-0.8431, -0.9529, -0.9373,  ..., -0.9608, -0.9373, -0.8275],
          ...,
          [ 0.4431,  0.7176,  0.7412,  ...,  0.7176,  0.6471,  0.6706],
          [ 0.5529,  0.8431,  0.8667,  ...,  0.8745,  0.8275,  0.8039],
          [ 0.5137,  0.8431,  0.8902,  ...,  0.9137,  0.8745,  0.7804]],

         [[-0.6784, -0.8118, -0.8275,  ..., -0.8431, -0.8196, -0.7020],
          [-0.8118, -0.9294, -0.9294,  ..., -0.9451, -0.9294, -0.8118],
          [-0.8431, -0.9529, -0.9373,  ..., -0.9608, -0.9373, -0.8275],
          ...,
          [ 0.4431,  0.7176,  0.7412,  ...,  0.7176,  0.6471,  0.6706],
          [ 0.5529,  0.8431,  0.8667,  ...,  0.8745,  0.8275,  0.8039],
          [ 0.5137,  0.8431,  0.8902,  ...,  0.9137,  0.8745,  0.7804]],

         [[-0.6784, -0.8118, -0.8275,  ..., -0.8431, -0.8196, -0.7020],
          [-0.8118, -0.9294, -

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

# Determine classes

In [12]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [13]:
print(classes)

['ClearSky', 'altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus']


# CNN Model

In [43]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=11):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.bn2=nn.BatchNorm2d(num_features=20)
        #Shape= (256,20,150,150)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output = output.view(output.size(0), -1)
            
            
        output=self.fc(output)
            
        return output
            
        


In [44]:
model=ConvNet(num_classes=11).to(device)

# Optimizer

In [45]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [46]:
num_epochs=10

In [47]:
train_count=len(glob.glob(train_path+'/**/*.jpeg')) + len(glob.glob(train_path+'/**/*.jpg')) + len(glob.glob(train_path+'/**/*.bmp')) + len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.jpeg')) + len(glob.glob(test_path+'/**/*.jpg')) + len(glob.glob(test_path+'/**/*.bmp')) + len(glob.glob(test_path+'/**/*.png'))

In [48]:
print(train_count,test_count)

1935 489


# Training

In [53]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.h5')
        best_accuracy=test_accuracy

tensor([ 34.4856,  37.0562,  39.6122,  43.3499,  55.5837,  71.9907,  39.5385,
         51.5728,  38.0160,  34.0378,  40.7788,  35.7200,  35.8805,  38.8621,
         48.9560,  46.8744,  34.2476,  39.2597,  32.7415, 101.3928,  37.0323,
         41.2571,  26.4325,  34.4298,  43.1226,  44.1093,  36.9646,  33.1498,
         28.7086,  31.3172,  40.1266,  33.1210], device='cuda:0')
tensor([31.8564, 34.0028, 35.8161, 23.3772, 44.1155, 32.3380, 40.3880, 44.6800,
        40.3635, 38.7195, 31.5263, 33.4547, 33.3887, 36.5734, 39.9145, 40.9472,
        40.9403, 35.4952, 44.7811, 38.5032, 37.4356, 27.5770, 43.7342, 40.4023,
        42.1794, 35.6628, 38.3837, 27.6866, 33.8342, 67.2977, 37.5290, 32.3195],
       device='cuda:0')
tensor([23.2065, 36.4322, 42.6175, 33.0609, 67.9043, 71.3681, 41.5422, 36.3031,
        50.4502, 49.8307, 62.7892, 50.1554, 46.9865, 32.9695, 30.8489, 34.4068,
        51.0989, 53.1348, 31.6879, 38.3366, 42.3152, 36.1076, 34.6977, 40.6596,
        35.6242, 34.9911, 34.3551, 36

KeyboardInterrupt: 